In [ ]:
import random
import numpy as np
import json
import os

from problem import TSPProblem, KnapsackProblem
from GA.crossover_operators import order_crossover, pmx_crossover, cycle_crossover, arithmetic_crossover, blend_crossover, sbx_crossover
from GA.mutation_operators import swap_mutation, inversion_mutation, insertion_mutation, gaussian_mutation, uniform_mutation, polynomial_mutation
from GA.selection_operators import tournament_selection, roulette_wheel_selection, truncation_selection
from experiment import Experiment, MFEAExperiment, NumpyEncoder

In [30]:
random.seed(42)
np.random.seed(42)
results = {}
seeds = range(30)

In [24]:
tsp = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='permutation')

crossover_ops = {
    "order": order_crossover,
    "pmx": pmx_crossover,
    "cycle": cycle_crossover,
}

mutation_ops = {
    "swap": swap_mutation,
    "inversion": inversion_mutation,
    "insertion": insertion_mutation
}

selection_ops = {
    "tournament": tournament_selection,
    "roulette": roulette_wheel_selection,
    "truncation": truncation_selection
}

task1 = {}
for crossover_name, crossover_op in crossover_ops.items():
    for mutation_name, mutation_op in mutation_ops.items():
        for selection_name, selection_op in selection_ops.items():
            exp = Experiment(
                problem=tsp,
                seeds=seeds,
                pop_size=100,
                generations=500,
                crossover_prob=0.9,
                mutation_prob=0.1,
                crossover_op=crossover_op,
                mutation_op=mutation_op,
                selection_op=selection_op,
                elitism=2
            )
            exp.run(verbose=False)
            task1[f"{crossover_name}_{mutation_name}_{selection_name}"] = -exp.stats["max"]
results["task1"] = task1
task1

Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp


Running tsp experiment:   0%|          | 0/30 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [32]:
tsp_real = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='real')

crossover_ops = {
    "arithmetic": arithmetic_crossover,
    "blend": blend_crossover,
    "sbx": sbx_crossover
}

mutation_ops = {
    "gaussian": gaussian_mutation,
    "uniform": uniform_mutation,
    "polynomial": polynomial_mutation
}

selection_ops = {
    "tournament": tournament_selection,
    "truncation": truncation_selection
}

not_match = ["arithmetic_uniform_tournament", "arithmetic_uniform_truncation"]

task2 = {}
for crossover_name, crossover_op in crossover_ops.items():
    for mutation_name, mutation_op in mutation_ops.items():
        for selection_name, selection_op in selection_ops.items():
            if f"{crossover_name}_{mutation_name}_{selection_name}" in not_match:
                continue
            exp = Experiment(
                problem=tsp_real,
                seeds=range(5),
                pop_size=100,
                generations=500,
                crossover_prob=0.9,
                mutation_prob=0.1,
                crossover_op=crossover_op,
                mutation_op=mutation_op,
                selection_op=selection_op,
                elitism=3
            )
            exp.run(verbose=False)
            task2[f"{crossover_name}_{mutation_name}_{selection_name}"] = -exp.stats["max"]
            print(f"{crossover_name}_{mutation_name}_{selection_name}: {-exp.stats['max']}")
results["task2"] = task2
task2

Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp


Running tsp experiment: 100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


✓ Results saved to results/ga/arithmetic/gaussian/tournament/tsp
arithmetic_gaussian_tournament: 591.4651356907568


Running tsp experiment: 100%|██████████| 5/5 [00:21<00:00,  4.36s/it]


✓ Results saved to results/ga/arithmetic/gaussian/truncation/tsp
arithmetic_gaussian_truncation: 601.459936766761


...

Running tsp experiment: 100%|██████████| 5/5 [00:10<00:00,  2.12s/it]


✓ Results saved to results/ga/sbx/polynomial/tournament/tsp
sbx_polynomial_tournament: 747.1163183587287


Running tsp experiment: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]

✓ Results saved to results/ga/sbx/polynomial/truncation/tsp
sbx_polynomial_truncation: 800.2532116904597


{'arithmetic_gaussian_tournament': np.float64(591.4651356907568),
 'arithmetic_gaussian_truncation': np.float64(601.459936766761),
 'arithmetic_polynomial_tournament': np.float64(631.3479547459739),
 'arithmetic_polynomial_truncation': np.float64(610.1503546744484),
 'blend_gaussian_tournament': np.float64(584.5829313960888),
 'blend_gaussian_truncation': np.float64(635.4403788296246),
 'blend_uniform_tournament': np.float64(918.5611747486735),
 'blend_uniform_truncation': np.float64(970.9248868642643),
 'blend_polynomial_tournament': np.float64(670.5495254805362),
 'blend_polynomial_truncation': np.float64(585.5478343017356),
 'sbx_gaussian_tournament': np.float64(742.7482144337768),
 'sbx_gaussian_truncation': np.float64(736.9664757510564),
 'sbx_uniform_tournament': np.float64(914.3577223264239),
 'sbx_uniform_truncation': np.float64(968.2228975873712),
 'sbx_polynomial_tournament': np.float64(747.1163183587287),
 'sbx_polynomial_truncation': np.float64(800.2532116904597)}

In [ ]:
tsp51 = TSPProblem(file_path="dataset/tsp/eil51.tsp", encoding='permutation')
tsp101 = TSPProblem(file_path="dataset/tsp/eil101.tsp", encoding='permutation')

mfea_exp = MFEAExperiment(
    problems=[tsp51, tsp101],
    seeds=seeds,
    pop_size=100,
    generations=500,
    rmp=0.3,
    cross_crossover_op=order_crossover
)
mfea_exp.run(verbose=False)

results["task3"] = {
    "tsp51": mfea_exp.stats["task_0"]["best_tour_length"],
    "tsp101": mfea_exp.stats["task_1"]["best_tour_length"]
}
mfea_exp.stats

Loaded TSP problem with 51 cities from dataset/tsp/eil51.tsp
Loaded TSP problem with 101 cities from dataset/tsp/eil101.tsp


Running MFEA on order_tournament_tsp_tsp: 100%|██████████| 30/30 [04:13<00:00,  8.45s/it]


{'task_0': {'mean': np.float64(-632.8180906523943),
  'std': np.float64(111.076266576941),
  'min': np.float64(-912.8997636490969),
  'max': np.float64(-482.15805138685903),
  'median': np.float64(-603.0773978547977),
  'best_tour_length': np.float64(482.15805138685903),
  'mean_tour_length': np.float64(632.8180906523943)},
 'task_1': {'mean': np.float64(-1086.108338987032),
  'std': np.float64(71.20454098866051),
  'min': np.float64(-1270.8937502371793),
  'max': np.float64(-952.7089389557199),
  'median': np.float64(-1080.9039494802712),
  'best_tour_length': np.float64(952.7089389557199),
  'mean_tour_length': np.float64(1086.108338987032)}}

In [ ]:
knapsack = KnapsackProblem(file_prefix="dataset/knapsack/p07", encoding='real')

mfea_exp2 = MFEAExperiment(
    problems=[tsp51, knapsack],
    seeds=seeds,
    pop_size=100,
    generations=500,
    rmp=0.3,
    cross_crossover_op=arithmetic_crossover
)
mfea_exp2.run(verbose=False)

results["task4"] = {
    "tsp51": mfea_exp2.stats["task_0"]["best_tour_length"],
    "knapsack": mfea_exp2.stats["task_1"]["max"]
}
mfea_exp2.stats

Loaded Knapsack: 15 items, capacity=750


Running MFEA on arithmetic_tournament_tsp_knapsack: 100%|██████████| 30/30 [01:51<00:00,  3.73s/it]


{'task_0': {'mean': np.float64(-536.8214916529745),
  'std': np.float64(31.20547015703557),
  'min': np.float64(-602.5267964950633),
  'max': np.float64(-471.3120525815979),
  'median': np.float64(-527.4790413014534),
  'best_tour_length': np.float64(471.3120525815979),
  'mean_tour_length': np.float64(536.8214916529745)},
 'task_1': {'mean': np.float64(1457.1666666666667),
  'std': np.float64(1.8089284734953512),
  'min': np.int64(1451),
  'max': np.int64(1458),
  'median': np.float64(1458.0)}}

In [ ]:
os.makedirs("results/simple", exist_ok=True)
with open("results/simple/results.json", "w") as f:
    json.dump(results, f, indent=4, cls=NumpyEncoder)